##### 마르코프 체인과 LSTM으로 문장 생성하기 

In [ ]:
# 마르코프 체인 : 확률을 기반으로 하는 방법 (=워드 샐러드)
#               기존 문장을 기반으로 문장을 자동으로 생성
#               과거의 상태를 무시하고, 현재의 상태만을 기반으로 다음 상태를 선택하는 것
#               단어의 실질적인 의미 연관성을 생각하지 않고 문장을 조합
#               문자 단위가 아닌 단어 단위로 처리 -> N-gram과의 차이

# 마르코프 체인으로 문장을 만드는 과정
# (1) 문장을 단어로 분할(형태소 분석)
# (2) 단어의 전후 연결을 딕셔너리에 등록
# (3) 딕셔너리를 사용해 임의의 문장을 생성

In [1]:
## 파이썬을 이용한 맞춤법 처리 (네이버 맞춤법은 서비스 종료됨)
# 참조 : https://github.com/spacedev-official/py-aiohanspell
# pip install py-aiohanspell
# pip install aiohttp : 서버 api와 통신해서 결과값을 가져오는 모듈

## 예시
from aiohanspell import spell_checker
result = await spell_checker.check(u'안녕 하세요. 저는 한국인 입니다. 한글로 작성헸습니다.')
result      # 1 : 맞춤법 오류, 2 : 띄어쓰기 오류
result.as_dict()    # 딕셔너리 형태로 조회 가능
result.checked      # 필요한 값만 추출 가능

'안녕하세요. 저는 한국인입니다. 한글로 작성했습니다.'

In [2]:
import codecs
from bs4 import BeautifulSoup
from konlpy.tag import Twitter, Okt
from aiohanspell import spell_checker

import os, re, json, random

# 마르코프 체인 딕셔너리 만들기
def make_dic(words) :
    tmp = ['@']
    dic = {}
    for word in words:
        tmp.append(word)
        if len(tmp) < 3 : continue
        if len(tmp) > 3 : tmp = tmp[1:]
        
        set_word3(dic, tmp)     # 딕셔너리에 데이터 등록하는 작업
        
        if word == '.' :
            tmp = ['@']
            continue
    return dic

# 딕셔너리에 데이터 등록
def set_word3(dic, s3):
    w1, w2, w3 = s3
    if not w1 in dic: dic[w1] = {}
    if not w2 in dic[w1]: dic[w1][w2] = {}
    if not w3 in dic[w1][w2]: dic[w1][w2][w3] = 0
    dic[w1][w2][w3] += 1
    
# 임의의 문장 생성
def make_sentence(dic) :
    ret = []
    if not '@' in dic : return 'no dic'
    top = dic['@']
    w1 = word_choice(top)
    w2 = word_choice(top[w1])
    ret.append(w1)
    ret.append(w2)
    while True :
        w3 = word_choice(dic[w1][w2])
        ret.append(w3)
        if w3 == '.' : break
        w1, w2 = w2, w3
    ret = ''.join(ret)
    
    return ret  # 맞춤법 검사는 메인에서 처리 예정

def word_choice(sel) :      # 랜덤으로 단어를 골라옴
    keys = sel.keys()
    return random.choice(list(keys))


########### 메인 ###########

### 문장 읽기
toji_file = 'toji.txt'
dict_file = 'markov-toji.json'
if not os.path.exists('data/'+dict_file) :
    # 토지 텍스트파일 불러오기
    fp = codecs.open('data/'+toji_file, 'r', encoding='utf-16')
    soup = BeautifulSoup(fp, 'html.parser')
    body = soup.select_one('body > text')
    text = body.getText()
    text = text.replace('…', '')
    
    # 형태소 분석
    twitter = Okt()
    malist = twitter.pos(text, norm=True)   # 형태소 단위로 분해
    words = []
    for word in malist :
        if not word[1] in ['Punctuation'] : # 구두점 제외시키기
            words.append(word[0])
        if word[0] == '.' :                 # 마침표는 포함시키기
            words.append(word[0])
    # 딕셔너리 생성
    dic = make_dic(words)   # 데이터를 저장한 딕셔너리 생성
    json.dump(dic, open('data/'+dict_file, 'w', encoding='utf-8'))
else :
    dic = json.load(open('data/'+dict_file, 'r'))
    

for i in range(3) :
    s = make_sentence(dic)
    t = await spell_checker.check(s)
    print(t.checked)
    print('-'*40)


민란에 가담하여 쫓기는 처지요 본시부터 잠 충이나 돼서 다시는 못 가게서성대는 김서방은 늙은 상전을 업고 최 참판 자손을 마구잡이로 해대었으나 함안댁 입에서는 긴 얘기가 나오기 마련이다.
----------------------------------------
요령 흔들리는 소리가 꺼진다.
----------------------------------------
연잎이 뜸한 수면에서는 소금쟁이가 뱅뱅이를 돈다.
----------------------------------------


In [4]:
from pprint import pprint
import json
dic = json.load(open('data/markov-toji.json', 'r'))

In [6]:
pprint(dic['민란'])       # 만들어낸 딕셔너리를 확인 가능

{'도': {'여기저기': 1}, '때': {'관가': 1}, '에': {'가담': 1}, '이': {'나야': 1, '난': 2}}


##### LSTM으로 문장 생성하기 

In [ ]:
# 문장 생성 알고리즘 :  재귀 신경망(Recurrent Neural Network / RNN) , LSTM(Long Short Term-Memory)

### LSTM : 바로 이전 데이터만 기억하는 RNN을 장기적으로 기억할 수 있게 개량한 알고리즘
#          시간 순서를 기반으로 문장을 쉽게 생성

In [34]:
import codecs
from bs4 import BeautifulSoup
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random, sys

fp = codecs.open('data/toji.txt', 'r', encoding='utf16')
soup = BeautifulSoup(fp, 'html.parser')
body = soup.select_one('body')
text = body.getText() + ' '
print('코퍼스의 길이 :', len(text))

# 문자를 하나씩 읽어서 ID 부여하기
chars = sorted(list(set(text)))
print('사용되고 있는 문자 수 :', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))    # 문자 -> ID
indices_char = dict((i, c) for i, c in enumerate(chars))    # ID -> 문자

# 텍스트를 maxlen개의 문자로 자르고 다음에 오는 문자 등록
maxlen = 20
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step) :
    sentences.append(text[i : i + maxlen])
    next_chars.append(text[i + maxlen])
print('학습할 구문의 수 :', len(sentences))
print('텍스트를 ID 벡터로 변환합니다~~!!')
X = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
y = np.zeros((len(sentences), len(chars)), dtype = np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices [next_chars[i]]] = 1

# 모델 구축하기(LSTM)
print('모델을 구축합니다~~!!')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars )))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss = 'categorical_crossentropy', optimizer = optimizer)

코퍼스의 길이 : 311682
사용되고 있는 문자 수 : 1692
학습할 구문의 수 : 103888
텍스트를 ID 벡터로 변환합니다~~!!


C:\Users\sein1\AppData\Local\Temp\ipykernel_27044\578283937.py:33: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  X = np.zeros((len(sentences), maxlen, len(chars)), dtype = np.bool)
C:\Users\sein1\AppData\Local\Temp\ipykernel_27044\578283937.py:34: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype = np.bo

모델을 구축합니다~~!!


C:\Users\sein1\AppData\Roaming\Python\Python310\site-packages\keras\optimizers\optimizer_v2\rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [35]:
# 후보를 배열에서 꺼내기
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

# 학습시키고 텍스트 생성하기 반복
for iteration in range(1, 5):
    print()
    print('-' * 50)
    print('=', iteration)
    model.fit(X, y, batch_size=128, epochs=1) # 임의의 시작 텍스트 선택하기
    start_index = random.randint(0, len(text) - maxlen - 1)
    # 다양한 다양성의 문장 생성
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('---- 다양성 = ', diversity)
        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('---- 시드 = "' + sentence + '"')
        sys.stdout.write(generated)
        
        # 시드를 기반으로 텍스트 자동 생성        
        for i in range(200):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.
            # 다음에 올 문자를 예측하기
            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]
            # 출력하기
            generated += next_char
            sentence = sentence[1:] + next_char
            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


--------------------------------------------------
= 1
812/812 [==============================] - 71s 86ms/step - loss: 3.8521

---- 다양성 =  0.2
---- 시드 = "로 뻗어온 학자였다. 그럼에도 불구하"
로 뻗어온 학자였다. 그럼에도 불구하고 있었다.
"그것 없이 그 말이요."
"그렇게 그것 없는 것 같다. 어디 그 사람들이 있었다.
"그것 없는 그것도 있는 것이다. 그 그 사람들은 그 사람들은 그 말이 있었다.
"아니 아니 아니 아니다. 그 사람 같은 그 말이 있었다.
"그렇지 않았소. 그 그 사람 없이 그 말이요."
"그렇게 그렇게 하는 것이다. 그 말이 있었다.
"그것 같이 있었다.
"

---- 다양성 =  0.5
---- 시드 = "로 뻗어온 학자였다. 그럼에도 불구하"
로 뻗어온 학자였다. 그럼에도 불구하게 없는 것도 없었다.
"당님  안 가디 아니지."
"그래 그 양반 없이 그렇지마요."
"일 되어 못 없이요."
"장치 아니 그 적에  그 자신 이 나는 것 같다. 그것도 있었다.
"아니지 손아무 없는 것이요."
"그래?"
"그 포신 없이 거 성님 하더라고 그 마도 있었다.
"와 자신 없는 일이 있는 것 같다. 하나 보는 것도 없는 도 머리를 하는 것도 없

---- 다양성 =  1.0
---- 시드 = "로 뻗어온 학자였다. 그럼에도 불구하"
로 뻗어온 학자였다. 그럼에도 불구하들면 없는 테라보고고 지나지를 목어재는 죽직이 았는데,"
"슴상에 동도 났 살어니다. 자분 있었을 타시같이는, 강포수는 김치수은 조금경요."
그래 은모새 그치인 달뿐겄었다.
"사내여낚 되는 문것 같지디 않여니다."
목과 쫓았던 것이어서 들어
서울본 져미리를 간었다.
"그놈선선이 디래 않았다마는 말이 있지 시섯른들이 더 나? 치수는 계집졌 녁에 있었다.
"없

---- 다양성 =  1.2
---- 시드 = "로 뻗어온 학자였다. 그럼에도 불구하"
로 뻗어온 학자였